In [11]:
import requests
import os
import shutil
import json
import pandas as pd
import yaml

import utils, request_models

import warnings
warnings.filterwarnings('ignore')


In [12]:
######## File paths ########
with open("../configs/self-driving-demo.yaml", "r") as f:
    config = yaml.safe_load(f)

protein_file_path = config['paths']['protein_file_path']
diffdock_output_dir = config['paths']['diffdock_output_dir']
dsmbind_input_dir = config['paths']['diffdock_output_dir']

starting_molecule_csv = config['paths']['starting_molecule_csv']
molmim_generated_csv = config['paths']['molmim_generated_csv']
dsmbind_predictions_csv = config['paths']['dsmbind_predictions_csv']

utils.prepare_output_directory(diffdock_output_dir)
utils.prepare_output_directory(dsmbind_input_dir)

# Get folded protein
folded_protein = utils.file_to_json_compatible_string(protein_file_path)

# Get starting molecules
df_starting_molecules = pd.read_csv(starting_molecule_csv)

# Round 0
print("Round 0")
molecule_name = df_starting_molecules['Molecules'][0]
molecule = df_starting_molecules['Smiles'][0]

# Molecular Generation with MolMIM
molmim_response = request_models.call_molmim(molecule)
generated_ligands = '\n'.join([v['smiles'] for v in molmim_response['generated']])
utils.update_dataframe_molmim_generated_molecules(molmim_response['generated'], molecule_name)

# Protein-Ligand Docking with DiffDock
diffdock_response = request_models.call_diffdock(folded_protein, generated_ligands)
utils.create_diffdock_outputs_dsmbind_inputs(molecule_name, diffdock_response)

# Binding Affinity with DSMBind
os.system("python /workspace/bionemo/examples/molecule/dsmbind/infer.py")


Round 0


/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
[NeMo W 2024-12-17 21:15:40 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/megatron/core/tensor_parallel/layers.py:257: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
      def forward(
    
[NeMo W 2024-12-17 21:15:40 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/megatron/core/tensor_parallel/layers.py:268: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
      def backward(ctx, grad_output):
    
[NeMo W 2024-12-17 21:15:40 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/megatron/core/tensor_parallel/layers.py:328: FutureWarni

[NeMo I 2024-12-17 21:15:47 megatron_hiddens:110] Registered hidden transform sampled_var_cond_gaussian at bionemo.model.core.hiddens_support.SampledVarGaussianHiddenTransform
[NeMo I 2024-12-17 21:15:47 megatron_hiddens:110] Registered hidden transform interp_var_cond_gaussian at bionemo.model.core.hiddens_support.InterpVarGaussianHiddenTransform


[NeMo W 2024-12-17 21:15:48 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/hydra/_internal/hydra.py:119: UserWarning: Future Hydra versions will no longer change working directory at job runtime by default.
    See https://hydra.cc/docs/1.2/upgrades/1.1_to_1.2/changes_to_job_working_dir/ for more information.
      ret = run_job(
    


[NeMo I 2024-12-17 21:15:48 infer:68] 
    
    ************** Experiment Configuration ***********
[NeMo I 2024-12-17 21:15:48 infer:69] 
    data:
      processed: false
      raw_inference_data_dir: ${oc.env:BIONEMO_HOME}/generative-virtual-screening/data/dsmbind_inputs
      predictions_csv: ${oc.env:BIONEMO_HOME}/generative-virtual-screening/data/dsmbind_predictions.csv
      processed_inference_data_path: ${oc.env:BIONEMO_HOME}/examples/tests/test_data/molecule/dsmbind/inference/processed.pkl
    model:
      hidden_size: 256
      mpn_depth: 3
      num_heads: 4
      dropout: 0.1
      threshold: 10.0
      vocab_size: 38
      aa_size: 21
      max_residue_atoms: 14
    inference:
      num_gpus: 1
      ckpt_path: ${oc.env:BIONEMO_HOME}/models/molecule/dsmbind/dsmbind.pth
    
[NeMo I 2024-12-17 21:15:48 infer:75] ************** Load Trained Model ***********
[NeMo I 2024-12-17 21:15:48 infer:82] # Model parameters: 1019136
[NeMo I 2024-12-17 21:15:48 infer:94] **************

  0%|          | 0/5 [00:00<?, ?it/s][NeMo W 2024-12-17 21:15:48 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'END' at line 2332
      warnings.warn(
    
Failed to patch pandas - unable to change molecule rendering
[NeMo W 2024-12-17 21:15:48 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'END' at line 2332
      warnings.warn(
    
Failed to patch pandas - unable to change molecule rendering
[NeMo W 2024-12-17 21:15:48 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'END' at line 2332
      warnings.warn(
    
Failed to patch pandas - unable to change molecule rendering
[NeMo W 2024-12-17 21:15:48 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrec

'Molecule'
[NeMo I 2024-12-17 21:15:48 infer:96] ************** Loading Inference Dataset ***********
[NeMo I 2024-12-17 21:15:48 infer:104] ************** Starting Inference ***********
[NeMo I 2024-12-17 21:15:49 infer:106] Predictions: [-1515169.5, -1443176.625, -1517427.75, -1491821.375]


0

In [3]:

df_molmim = pd.read_csv(molmim_generated_csv) 
df_dsmbind = pd.read_csv(dsmbind_predictions_csv)
df_joined = pd.concat([df_molmim, df_dsmbind], axis=1)

df_joined.to_csv('../data/results.csv')

In [20]:
def update_dataframe_molmim_generated_molecules(molmim_generated, 
                                                starting_molecule_name, 
                                                molmim_generated_csv = molmim_generated_csv):
    import pandas as pd

    df = pd.DataFrame(molmim_generated)
    # Reset the index and make it a column
    df.reset_index(inplace=True)
    df.rename(columns={'smiles':'generated_smiles',
                    'score':'molmim_qed_score',
                    'index':'generated_compound_index'},
                    inplace=True)
    df['starting_molecule'] = starting_molecule_name

    if os.path.exists(molmim_generated_csv):
        print("The CSV file exists.")
        df_old = pd.read_csv(molmim_generated_csv)
        df_merged = pd.concat([df_old, df], ignore_index=True)

        df_merged.to_csv(molmim_generated_csv, index=False)
    else:
        df.to_csv(molmim_generated_csv, index=False)

        

    

In [21]:
rd = 0
if len(df_joined[df_joined['DSMBind_predictions'] < -1600000]) < 5:
    rd = rd + 1
    print(f"Round {rd}")

    molecule_name = df_starting_molecules['Molecules'][rd]
    molecule = df_starting_molecules['Smiles'][rd]

    # Molecular Generation with MolMIM
    print("MolMIM...")
    molmim_response = request_models.call_molmim(molecule)
    generated_ligands = '\n'.join([v['smiles'] for v in molmim_response['generated']])
    update_dataframe_molmim_generated_molecules(molmim_response['generated'], molecule_name)

    # Protein-Ligand Docking with DiffDock
    print("DiffDock...")
    diffdock_response = request_models.call_diffdock(folded_protein, generated_ligands)
    utils.create_diffdock_outputs_dsmbind_inputs(molecule_name, diffdock_response)

    # Binding Affinity with DSMBind
    print("DSMBind...")
    os.system("python /workspace/bionemo/examples/molecule/dsmbind/infer.py")

    df_molmim = pd.read_csv(molmim_generated_csv) 
    df_dsmbind = pd.read_csv(dsmbind_predictions_csv)
    df_joined = pd.concat([df_molmim, df_dsmbind], axis=1)

    df_joined.to_csv('../data/results.csv')


Round 1


KeyboardInterrupt: 

In [17]:
molmim_response['generated']

[{'smiles': 'CC(C)(C)C(NCc1cnn(CC(F)F)c1)C(C)(C)C',
  'score': 0.8946356255903143},
 {'smiles': 'CC(C)(C)C(NC(=O)C(F)(F)F)C(C)(C)C(=O)N1C[C@@H]2[C@@H](C1)C2(C)C',
  'score': 0.8380480056463188},
 {'smiles': 'CC(C)(C)C(NC(=O)C(F)(F)F)C(=O)N1CC2(CCOC2)C1',
  'score': 0.8342971068120115},
 {'smiles': 'CC(C)(C)C(NC(=O)C(F)(F)F)C(=O)N1CC2(CCOC2)C1',
  'score': 0.8342971068120115},
 {'smiles': 'CC(C)(C)C(NC(=O)C(F)(F)F)C(=O)N1CC2(CCOC2)C1',
  'score': 0.8342971068120115}]

In [18]:
df_tmp = pd.DataFrame()
df_tmp = molmim_response['generated']
df_tmp

[{'smiles': 'CC(C)(C)C(NCc1cnn(CC(F)F)c1)C(C)(C)C',
  'score': 0.8946356255903143},
 {'smiles': 'CC(C)(C)C(NC(=O)C(F)(F)F)C(C)(C)C(=O)N1C[C@@H]2[C@@H](C1)C2(C)C',
  'score': 0.8380480056463188},
 {'smiles': 'CC(C)(C)C(NC(=O)C(F)(F)F)C(=O)N1CC2(CCOC2)C1',
  'score': 0.8342971068120115},
 {'smiles': 'CC(C)(C)C(NC(=O)C(F)(F)F)C(=O)N1CC2(CCOC2)C1',
  'score': 0.8342971068120115},
 {'smiles': 'CC(C)(C)C(NC(=O)C(F)(F)F)C(=O)N1CC2(CCOC2)C1',
  'score': 0.8342971068120115}]